In [97]:
from db import ohlcv_db, posts_db
from typing import List,Set,Iterable
import pymongo
import pprint
import pandas as pd
from datetime import datetime, timedelta

In [98]:
def get_top_pairs(max_num:int=100,last_hours:int=1) -> List:
    """
        Function to return at most the top max_num distinct pairs by compound volume from ohlcv db in the last last_hours
    """
    ohlcv_documents = ohlcv_db.aggregate(
            [
                {
                    "$match": {
                        "timestamp": {
                            "$gte": datetime.utcnow() - timedelta(hours=last_hours)
                        },
                    }
                },
                {
                    "$group": {
                        "_id": {"$concat": ["$pair_symbol", "-", "$pair_base"]},
                        "volume_sum": {"$sum": {"$toDouble": "$volume"}},
                    }
                },

                {"$sort": {"volume_sum": pymongo.DESCENDING}},
                {"$limit": max_num},
            ],
        )
    top_pairs_by_volume = [x['_id'].upper() for x in ohlcv_documents]

    return top_pairs_by_volume

In [99]:
def get_latest_posted_pairs(top_pairs:List,max_num:int=5) -> Set:
    """
        Function to return at most the top max_num distinct pairs by latest post time from posts db
    """

    posts_documents = posts_db.aggregate(
        [
            {
                "$match": {
                    "pair": {"$in": top_pairs},
                }
            },
            {
                "$group": {
                    "_id": "$pair",
                    "time": {"$max": "$time"},
                }
            },
            {"$sort": {"time": pymongo.DESCENDING}},
            {"$limit": max_num},
        ],
    )
    latest_posted_pairs = {x['_id'].upper() for x in posts_documents}

In [100]:
def get_latest_posted_pair() -> str:
    
    document = posts_db.find({}).sort('time', pymongo.DESCENDING).limit(1)
    return document[0].get('pair')

In [101]:
def get_pair_to_post(top_pairs:List,lastest_posted_pairs_among_top:Iterable = None) -> str:
    """
        Function to return the pair to post
    """
    if (lastest_posted_pairs_among_top is None) or len(lastest_posted_pairs_among_top) == 0:
        # the first pair by volume
        return top_pairs[0]

    last_posted = get_latest_posted_pair()
    for pair in top_pairs:
        if pair in lastest_posted_pairs_among_top and not pair == last_posted:
            # Choose the first pair by volume among those oldest posted pairs
            return pair
    else:
        # Or the first pair by volume
        return top_pairs[0]

In [102]:

def get_message_dict(pair:str=None,pair_symbol:str=None,pair_base:str=None) -> dict:
    """
        Function to return a dictionary of marketVenue:volume percentage for a given pair
    """
    if pair is None and (pair_symbol is None or pair_base is None):
        raise ValueError("Either pair or pair_symbol and pair_base must be provided")

    if pair is not None:
        try:
            pair_symbol = pair.split('-')[0].lower()
            pair_base = pair.split('-')[1].lower()
        except:
            raise ValueError("Invalid pair provided, must be in the format 'SYMBOL-BASE")
            
    ohlcv_documents_for_pair = ohlcv_db.find({'pair_symbol':pair_symbol,'pair_base':pair_base}).sort('timestamp', pymongo.DESCENDING)

    latest_date = None
    message_dict = {
        'others':0
    }
    for item in ohlcv_documents_for_pair:
        
        if latest_date is None:
            latest_date = item['timestamp']
        if item['timestamp'] == latest_date:
            # pprint.pprint(item)
            if item['marketVenue'] not in message_dict:
                if len(message_dict) <= 5:
                    # key for top 5 marketVenues
                    message_dict[item['marketVenue']] = float(item['volume'])
                else:
                    # store the rest in 'others'
                    message_dict['others'] += float(item['volume'])
                continue    
            message_dict[item['marketVenue']] += float(item['volume'])
        else:
            break
    total_volume = sum(message_dict.values())
    message_dict = {key:round((value/total_volume)*100,2) for key,value in message_dict.items()}
    return message_dict

In [103]:
def compose_message(pair:str,message_dict:dict) -> str:
    """
        Function to compose a message for a given pair and message_dict
    """
    message = f"Top 5 marketVenues for {pair}:\n"
    for marketVenue,percentage in sorted(message_dict.items()):
        message += f"{marketVenue.capitalize()}: {percentage}%\n"
    return message

In [104]:

top_100_pairs_by_volume = get_top_pairs(max_num=100)
latest_posted_pair_among_top = get_latest_posted_pairs(top_pairs=top_100_pairs_by_volume,max_num=5)
pair_to_post = get_pair_to_post(top_100_pairs_by_volume,latest_posted_pair_among_top)
message_dict = get_message_dict(pair=pair_to_post)
message = compose_message(pair_to_post,message_dict)

In [110]:
print(top_100_pairs_by_volume)
message

['SHIB-USDT', 'SHIB-USD', 'SHIB-USDC', 'DOGE-USDT', 'XRP-USDT', 'ADA-USDT', 'BUSD-USDT', 'DOGE-USD', 'XLM-USDT', 'MATIC-USDT', 'BTC-USD', 'USDT-USD', 'EOS-USDT', 'ETH-USD', 'ADA-USD', 'USDC-USDT', 'XRP-USD', 'XLM-USD', 'LINK-USDT', 'ETH-USDT', 'BTC-USDT', 'DOT-USDT', 'DOGE-USDC', 'MATIC-USD', 'UNI-USDT', 'USDC-USD', 'LTC-USDT', 'CRO-USDT', 'AVAX-USDT', 'CRO-USD', 'LTC-USD', 'USDT-USDC', 'LINK-USD', 'EOS-USD', 'BNB-USDT', 'DOT-USD', 'UNI-USD', 'XLM-USDC', 'XRP-USDC', 'CRO-USDC', 'MATIC-USDC', 'AVAX-USD', 'ADA-USDC', 'BUSD-USD', 'XMR-USDT', 'BNB-USD', 'XMR-USD', 'DOT-USDC', 'UNI-USDC', 'LTC-USDC', 'ETH-USDC', 'LINK-USDC', 'AVAX-USDC', 'BNB-USDC', 'BUSD-USDC', 'EOS-USDC', 'XMR-USDC', 'BTC-USDC']


'Top 5 marketVenues for SHIB-USDT:\nBinance-us: 0.03%\nGateio: 0.97%\nHitbtc: 2.31%\nHuobi: 3.55%\nOkx: 37.8%\nOthers: 55.35%\n'

In [113]:
ohlcv_db_all = ohlcv_db.aggregate(
            [
                {
                    "$match": {
                        "timestamp": {
                            "$gte": datetime.utcnow() - timedelta(hours=1)
                        },
                    }
                },
                {
                    "$group": {
                        "_id": {"$concat": ["$pair_symbol", "-", "$pair_base"]},
                        "volume_sum": {"$sum": {"$toDouble": "$volume"}},
                    }
                },

                {"$sort": {"volume_sum": pymongo.DESCENDING}},
                {"$limit": -1},
            ],
        )

OperationFailure: invalid argument to $limit stage: Expected a non-negative number in: $limit: -1, full error: {'ok': 0.0, 'errmsg': 'invalid argument to $limit stage: Expected a non-negative number in: $limit: -1', 'code': 5107201, 'codeName': 'Location5107201', '$clusterTime': {'clusterTime': Timestamp(1672327075, 6), 'signature': {'hash': b"N/9\xe3/\xc1\x9ea?\x0e'\xf0\xc0_c\xc1\xbfO\xcc\xf8", 'keyId': 7172925401017090053}}, 'operationTime': Timestamp(1672327075, 6)}

In [107]:
print(datetime.now().astimezone())
print(datetime.utcnow().strftime("%Y-%m-%d %H:%M"))

2022-12-29 20:12:52.090602+05:00
2022-12-29 15:12


In [108]:
result =ohlcv_db.aggregate(
            [
                # filter for the last hour
                {
                    "$match": {
                        "timestamp": {
                            "$gte": datetime.utcnow() - timedelta(hours=1)
                        },
                    }
                },
                {
                    "$group": {
                        "_id": {"$concat": ["$pair_symbol", "-", "$pair_base"]},
                        "volume_sum": {"$sum": {"$toDouble": "$volume"}},
                    }
                },

                {"$sort": {"volume_sum": pymongo.DESCENDING}},
                {"$limit": 100},
            ],
        )

In [109]:
for r in result:
    pprint.pprint(r)

{'_id': 'shib-usdt', 'volume_sum': 981642359609.8337}
{'_id': 'shib-usd', 'volume_sum': 126840536652.51683}
{'_id': 'shib-usdc', 'volume_sum': 2304759115.8341}
{'_id': 'doge-usdt', 'volume_sum': 695248489.4782574}
{'_id': 'xrp-usdt', 'volume_sum': 94593524.24268238}
{'_id': 'ada-usdt', 'volume_sum': 39532602.97954555}
{'_id': 'busd-usdt', 'volume_sum': 32386426.0262}
{'_id': 'doge-usd', 'volume_sum': 22752256.11257993}
{'_id': 'xlm-usdt', 'volume_sum': 17433037.416644137}
{'_id': 'matic-usdt', 'volume_sum': 14830388.2605341}
{'_id': 'btc-usd', 'volume_sum': 12059557.8401752}
{'_id': 'usdt-usd', 'volume_sum': 7952076.90527107}
{'_id': 'eos-usdt', 'volume_sum': 7546637.159337027}
{'_id': 'eth-usd', 'volume_sum': 5093733.85332148}
{'_id': 'ada-usd', 'volume_sum': 4590460.39497463}
{'_id': 'usdc-usdt', 'volume_sum': 3152435.356908377}
{'_id': 'xrp-usd', 'volume_sum': 2791893.39322974}
{'_id': 'xlm-usd', 'volume_sum': 2542265.98476466}
{'_id': 'link-usdt', 'volume_sum': 2115094.5127185555}
